In [1]:
!pip install pinecone-client

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install --upgrade --quiet  langchain-google-genai

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [3]:
!pip install -U langchain-community

Defaulting to user installation because normal site-packages is not writeable
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached httpx_sse-0.4.0-py3-none-any.whl.metadata (9.0 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Using cached mypy_extensions-1.0.0-py3-none-any.whl.metadata (1.1 kB)
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
    --------------------------------------- 0.0/2.5 MB 991.0 kB/s eta 0:00:03
   ---- ----------------------------------- 0.3/2.5 MB 3.0 MB/s eta 0:00:01
   -------- ------------------------------- 0.5/2.5 MB 3.7 MB/s eta 0:00:01
   ------------ --------------------------- 0.8/2.5 MB 4.6 MB/s eta 0:00:01
   ----------------- ---------------------- 1.1/2.5 MB 4.6 MB/s eta 0:00:01
   --------------------- ------------------ 1.3/2.5 MB 4.7 MB/s eta 0:00:01
   ------------------------ --------------- 1

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [6]:
import pinecone
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [7]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [5]:
def read_doc(directory):
    file_loader=PyPDFDirectoryLoader(directory)
    documents=file_loader.load()
    return documents

In [7]:
!pip install pypdf

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/298.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/298.0 kB ? eta -:--:--
   - -------------------------------------- 10.2/298.0 kB ? eta -:--:--
   -------- ------------------------------ 61.4/298.0 kB 656.4 kB/s eta 0:00:01
   ----------------------- ---------------- 174.1/298.0 kB 1.5 MB/s eta 0:00:01
   ---------------------------------------- 298.0/298.0 kB 1.8 MB/s eta 0:00:00


In [8]:
doc=read_doc('/content/sample_data/pdfs')
len(doc)


0

In [9]:
def chunk_data(docs,chunk_size=500,chunk_overlap=50):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)
    doc=text_splitter.split_documents(docs)
    return docs

In [10]:
documents=chunk_data(docs=doc)
len(documents)

0

In [6]:
from pinecone import Pinecone

pc = Pinecone(api_key="pcsk_4eZMH3_A9UiZwxMZNXuwkWbuSRhKLUtdjuxoaRNwNcQ6gMDyC1SxMxLvXhKEYQQF6BE5Q8")
index = pc.Index("oet-partc-content")

In [12]:
!pip install langchain-pinecone

Defaulting to user installation because normal site-packages is not writeable
  Using cached langchain_pinecone-0.2.0-py3-none-any.whl.metadata (1.7 kB)
Using cached langchain_pinecone-0.2.0-py3-none-any.whl (11 kB)
   ---------------------------------------- 0.0/370.8 kB ? eta -:--:--
   ---------------------------------------- 0.0/370.8 kB ? eta -:--:--
   - -------------------------------------- 10.2/370.8 kB ? eta -:--:--
   ---- ---------------------------------- 41.0/370.8 kB 393.8 kB/s eta 0:00:01
   ---------------- ----------------------- 153.6/370.8 kB 1.1 MB/s eta 0:00:01
   ---------------------------------------  368.6/370.8 kB 2.3 MB/s eta 0:00:01
   ---------------------------------------- 370.8/370.8 kB 1.9 MB/s eta 0:00:00
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.11.7
    Uninstalling aiohttp-3.11.7:
      Successfully uninstalled aiohttp-3.11.7


In [8]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)


In [14]:
vector_store.add_documents(documents)

[]

In [3]:
def retrieve_context(query,k=40):
    matching_results=vector_store.similarity_search(query,k=k)
    return matching_results

In [1]:
def retrieve_prompt(topic):
    # Retrieve documents based on the query
    doc_search = retrieve_context(topic)

    # Concatenating page content of documents
    documents_text = "\n".join([doc.page_content for doc in doc_search])

    # Prompt for generating OET Part A reading task in markdown format with complex numerical data
    prompt =  f"""
        Generate a detailed and professional reading passage for the Occupational English Test (OET) Part C reading section based on the topic '{topic}' as a mock test, adhering strictly to OET standards.

        The content should be derived from the following documents and reflect their key points:

        {documents_text}

        The passage should:
        - Begin with a main heading that is relevant to the topic.
        - Present the information in paragraph form without any subheadings.
        - Use advanced medical terminology and complex English language, suitable for healthcare professionals.
        - Be at least 2000 words in length and maintain a clinical and professional tone throughout.
        - Avoid introductory or concluding remarks such as "Here is a passage for..." or "I hope this helps...".
        - Include challenging and contextually rich content with a focus on critical thinking.
        - Maintain accuracy and relevance, using key insights from the provided documents.
        """


    return prompt



In [9]:
topic = "Conjunctive group therapy: a case study of an adult type I diabetes mellitus patient"
input = retrieve_prompt(topic)

In [10]:
from groq import Groq

# Replace with your actual API key
api_key = "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"

client = Groq(api_key=api_key)

# Example usage with client
chat_completion = client.chat.completions.create(
    messages=[{
        "role": "user",
        "content": f"{input}"
    }],
    model="llama3-8b-8192",
)

response_next_input=chat_completion.choices[0].message.content

print(chat_completion.choices[0].message.content)

**Conjunctive Group Therapy: A Case Study of an Adult Type I Diabetes Mellitus Patient**

The present case study demonstrates the efficacy of conjunctive group therapy in facilitating the self-management skills of an adult patient with type I diabetes mellitus (T1DM). Within the context of a multidisciplinary healthcare setting, conjunctive group therapy was employed as a supplementary therapeutic approach to enhance the patient's ability to cope with the psychological and emotional burdens associated with T1DM.

The patient, a 35-year-old male, was diagnosed with T1DM at the age of 25. Despite undergoing intensive education on insulin therapy and lifestyle modifications, he continued to experience significant difficulties in managing his condition. Specifically, he reported feelings of anxiety and depression when required to inject insulin, as well as struggles with adhering to a strict diet and exercise regimen. Moreover, the patient's relationships with family and friends had become

In [11]:
prompt_next = f"""
Generate **one task** with 6 multiple-choice questions (MCQs) based on the given context for an OET Reading Test Part C. Each question should have 4 options (1 correct answer, 3 distractors) following OET standards.

### Context:
{response_next_input}

### Output Format:
```json
{{
  "task": {{
    "questions": [
      {{
        "question": "First question based on the passage.",
        "options": ["a) Option A", "b) Option B", "c) Option C", "d) Option D"],
        "correct_answer": "A"
      }},
      {{
        "question": "Second question based on the passage.",
        "options": ["a) Option A", "b) Option B", "c) Option C", "d) Option D"],
        "correct_answer": "B"
      }},
      ...# 3 more
    ]
  }}
}}
"""


In [12]:
import json
chat_completion = client.chat.completions.create(
            messages=[{
                "role": "system",
                "content": (
                    "You are a helpful assistant. Please provide the output strictly in JSON format. "
                    "Do not include any extra text or explanations. Ensure the JSON is valid and well-structured. "
                    f"{prompt_next}"
                )
            }],
            model="llama-3.1-8b-instant",
        )
json_output = None 
response_next_input = chat_completion.choices[0].message.content

start_index = response_next_input.find('{')
end_index = response_next_input.rfind('}') + 1

# Extract and parse the JSON
json_text = response_next_input[start_index:end_index]


try:
    json_output = json.loads(json_text)
    print(json.dumps(json_output, indent=4))  # Pretty print the JSON
except json.JSONDecodeError as e:
    print("Invalid JSON response:", response_next_input)
    print("Error:", e)


{
    "task": {
        "questions": [
            {
                "question": "What type of therapy was employed as a supplementary approach to enhance the patient's ability to manage his condition?",
                "options": [
                    "a) Cognitive-behavioral therapy (CBT)",
                    "b) Conjunctive group therapy",
                    "c) Intensive education on insulin therapy and lifestyle modifications",
                    "d) Peer-facilitated therapy"
                ],
                "correct_answer": "b) Conjunctive group therapy"
            },
            {
                "question": "The patient's relationships with family and friends became strained due to which factor?",
                "options": [
                    "a) High blood sugar levels",
                    "b) Insulin therapy",
                    "c) The perceived burdens placed upon them by his condition",
                    "d) Strict diet and exercise regimen"
                ]